# Guided Exercise: Performance
#### Goals 🎯
In this tutorial, you will use TruEra to train and ingest a model, and then make performance improvements to our model in a structured and methodical way!

In this tutorial, you will:
1. Set up and view the results of performance and feature importance tests.
2. Find actionable issues with the model.
3. Mitigate these issues and re-upload your model to TruEra.
4. Retest the new model and confirm the effectivenesss of the mitigation strategy.

### First, set the credentials for your TruEra deployment.

If you don't have credentials yet, get them instantly by signing up for the free open beta: https://app.truera.net

In [ ]:
#connection details
TRUERA_URL = "https://app.truera.net"
AUTH_TOKEN = ""

### Install required packages for running in colab

In [ ]:
! pip install --upgrade shap
! pip install --upgrade xgboost==1.6.2
! pip install --upgrade truera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### From here, run the rest of the notebook and follow the analysis.

### Import packages, set up your TruEra Workspace

In [ ]:
import pandas as pd
import xgboost as xgb
import logging

from truera.client.truera_workspace import TrueraWorkspace
from truera.client.truera_authentication import TokenAuthentication

auth = TokenAuthentication(AUTH_TOKEN)
tru = TrueraWorkspace(TRUERA_URL, auth)

# set our environment to local compute so we can compute predictions and feature influences on our local machine
tru.set_environment("local")
# note: we'll periodically toggle between local and remote so we can interact with our remote deployment as well.

### Load the data and train an xgboost model
A bit about the model and data... 

In this example, we will use real data on the AirBnb listings 🏠 in San Francisco and Seattle to predict the listing price. The Airbnb data was scraped by Inside Airbnb and hosted by OpenDataSoft. Pricing a rental property is a challenging task for Airbnb owners as they need to understand the market, the features of their property, and how those features contribute to listing price.

You can find more information about the data here:
https://data.opendatasoft.com/explore/dataset/airbnb-listings%40public/

In [ ]:
# load data
san_francisco = pd.read_csv('https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-performance/San_Francisco.csv')
seattle = pd.read_csv('https://truera-examples.s3.us-west-2.amazonaws.com/data/starter-performance/Seattle.csv')

#make all float
san_francisco = san_francisco.astype(float)
seattle = seattle.astype(float)

#add point ids
sf_ids = [f"point_{i}" for i in range(len(san_francisco))]
san_francisco["id"] = sf_ids

se_ids = [f"point_{i}" for i in range(len(seattle))]
seattle["id"] = se_ids

# train first model
xgb_reg = xgb.XGBRegressor(eta = 0.2, max_depth = 4)
xgb_reg.fit(san_francisco.drop(['id','price'], axis = 1), san_francisco.price)

# create the first project
project_name = "Starter Example Companion - Performance"
tru.add_project(project_name, score_type = 'regression')

# add data collection
tru.add_data_collection("Data Collection v1")

# add data splits to the collection we just created
tru.add_data_split("San Francisco", pre_data = san_francisco, label_col_name="price", id_col_name = "id", split_type = "train")
tru.add_data_split("Seattle", pre_data = seattle, label_col_name="price", id_col_name = "id", split_type = "test")

# register the model
tru.add_python_model("model_1", xgb_reg, train_split_name="San Francisco", train_parameters = {"model_type":"xgb.XGBRegressor", "eta":0.2, "max_depth":4})

# sync with remote
tru.upload_project(upload_error_influences = False)

# Note that we're explicitly opting not to compute and upload error influences to optimize for ingestion speed. This is because SHAP is not optimized to compute error influences for regression models.
# Doing so will disable the "Debug" tab under Performance, and the "Contributors to Error Influence" tab under Drift.
# If you'd like to enable these pages, please remove the flag, i.e. run the method below. Please be aware that doing so will greatly increase the time required for ingestion.
# tru.upload_project()

INFO:truera.client.local.local_truera_workspace:Data collection in local environment is now set to "Data Collection v1". 
INFO:truera.client.local.local_truera_workspace:Data split "San Francisco" is added to local data collection "Data Collection v1", and set as the data split for the workspace context.
INFO:truera.client.local.local_truera_workspace:Data split "Seattle" is added to local data collection "Data Collection v1", and set as the data split for the workspace context.
INFO:truera.client.local.local_truera_workspace:Model "model_1" is added and associated with local data collection "Data Collection v1". "model_1" is set as the model for the workspace context.
INFO:truera.client.truera_workspace:Uploading data collection Data Collection v1.
INFO:truera.client.truera_workspace:Uploading data split San Francisco.


Uploading tmp3w4vvwxl.parquet -- ### -- file upload complete.
Put resource done.
Uploading tmpah5_spz7.parquet -- ### -- file upload complete.
Put resource done.


INFO:truera.client.remote_truera_workspace:Call to join: rowsets ['f52cf8c9-d3e7-4639-ac67-3f4f4356ccac', '51d42088-976c-48dc-a1ce-07d7abbfc2ca'] on ['id'] with default inner join.
INFO:truera.client.truera_workspace:Uploading data split Seattle.


Uploading tmp_5ekug69.parquet -- ### -- file upload complete.
Put resource done.
Uploading tmpdg9ge6yy.parquet -- ### -- file upload complete.
Put resource done.


INFO:truera.client.remote_truera_workspace:Call to join: rowsets ['32701444-217f-4923-abc8-4fbb384d38b1', 'b26cc7d0-f21d-4809-b299-fd3058af2386'] on ['id'] with default inner join.
INFO:truera.client.remote_truera_workspace:Setting remote model context to "model_1".


...

INFO:truera.client.remote_truera_workspace:Waiting for data split to materialize...
INFO:truera.client.truera_workspace:Influence algorithm for local project is: shap


Uploading tmpkj56n7smparquet -- ### -- file upload complete.
Put resource done.


INFO:truera.client.remote_truera_workspace:Waiting for data split to materialize...
INFO:truera.client.truera_workspace:Influence algorithm for local project is: shap


Uploading tmp744iczfqparquet -- ### -- file upload complete.
Put resource done.


INFO:truera.client.remote_truera_workspace:Waiting for data split to materialize...


### Issue: Overfitting

We observe there to be a large discrepency between our train and test accuracy!

In [ ]:
# toggle to remote to interact with the tester
tru.set_environment("remote")
tru.set_project(project_name)

# set default performance metric
tru.set_default_performance_metrics(["MAE"])

tru.set_data_collection("Data Collection v1")
tru.set_model("model_1")

train_split_name = "San Francisco"
test_split_name = "Seattle"

# generate the explainer and compute performance
explainer = tru.get_explainer(test_split_name, comparison_data_splits=[train_split_name])
explainer.compute_performance(metric_type="MAE")

INFO:truera.client.remote_truera_workspace:Data collection in remote environment is now set to "Data Collection v1". 
INFO:truera.client.remote_truera_workspace:Setting remote model context to "model_1".


,Split,MAE
0,Seattle,110.289665
1,San Francisco,47.496475


### Create Tests

To help us keep track of this issue, let's add a test for it!

Additionally, too many unimportant features is a common cause of overfitting. We should test for that as well.

Note that we could also set up tests for fairness and stability.

In [ ]:
# add performance tests
tru.tester.add_performance_test(
    test_name='Relative MAE Test',
    all_data_collections=True,
    data_split_name_regex='Seattle',
    metric="MAE",
    reference_split_name=train_split_name,
    fail_if_greater_than=0.80,
    fail_threshold_type="RELATIVE"
)

tru.tester.add_performance_test(
    test_name='RMSE Test',
    all_data_collections=True,
    data_split_name_regex='.*',
    metric="RMSE",
    fail_if_greater_than=115,
    fail_threshold_type="ABSOLUTE"
)

### View Test Results

In [ ]:
# get model results
tru.set_model("model_1")
tru.tester.get_model_test_results(test_types=["performance"])

,Name,Split,Segment,Metric,Score,Navigate
❌,Relative MAE Test,Seattle,ALL POINTS,MAE,110.2897,Explore in UI
❌,RMSE Test,Seattle,ALL POINTS,RMSE,154.0362,Explore in UI
✅,RMSE Test,San Francisco,ALL POINTS,RMSE,75.6326,Explore in UI


### Both tests are failing.

### From here, navigate to the TruEra Web App for analysis or continue on to Part 2!     [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16DexGCY1i4A5fLJZXC7xHPpqCSrQhVab)